In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import re
import os
import json
import platform
import datetime as dt
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

try:
    import pandas as pd
except ImportError:
    pd = None

# -------------------- 缓存路径 --------------------
CACHE_DIR = os.path.join(os.path.expanduser("~"), ".trade_parser_cache")
CACHE_FILE = os.path.join(CACHE_DIR, "mappings.json")

def ensure_cache_dir():
    os.makedirs(CACHE_DIR, exist_ok=True)

def _read_cache_file() -> dict:
    if not os.path.exists(CACHE_FILE):
        return {}
    try:
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return {}

def _write_cache_file(data: dict):
    ensure_cache_dir()
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def save_mappings_cache():
    data = _read_cache_file()
    data["ACCOUNT_NAME_MAP"] = ACCOUNT_NAME_MAP
    data["EXCHANGE_MAP"] = EXCHANGE_MAP
    data["CONTRACT_CN_NAME_MAP"] = CONTRACT_CN_NAME_MAP
    data["COMMODITY_CN_MAP"] = COMMODITY_CN_MAP
    _write_cache_file(data)

def load_mappings_cache() -> bool:
    data = _read_cache_file()
    if not data:
        return False
    ACCOUNT_NAME_MAP.clear(); ACCOUNT_NAME_MAP.update(data.get("ACCOUNT_NAME_MAP", {}))
    EXCHANGE_MAP.clear(); EXCHANGE_MAP.update(data.get("EXCHANGE_MAP", {}))
    CONTRACT_CN_NAME_MAP.clear(); CONTRACT_CN_NAME_MAP.update(data.get("CONTRACT_CN_NAME_MAP", {}))
    COMMODITY_CN_MAP.clear(); COMMODITY_CN_MAP.update(data.get("COMMODITY_CN_MAP", {}))
    return any([ACCOUNT_NAME_MAP, EXCHANGE_MAP, CONTRACT_CN_NAME_MAP, COMMODITY_CN_MAP])

def clear_account_cache():
    data = _read_cache_file()
    data["ACCOUNT_NAME_MAP"] = {}
    _write_cache_file(data)
    ACCOUNT_NAME_MAP.clear()

def clear_exchange_cache():
    data = _read_cache_file()
    data["EXCHANGE_MAP"] = {}
    data["CONTRACT_CN_NAME_MAP"] = {}
    _write_cache_file(data)
    EXCHANGE_MAP.clear()
    CONTRACT_CN_NAME_MAP.clear()

def clear_commodity_cache():
    data = _read_cache_file()
    data["COMMODITY_CN_MAP"] = {}
    _write_cache_file(data)
    COMMODITY_CN_MAP.clear()

def clear_all_cache():
    if os.path.exists(CACHE_FILE):
        try:
            os.remove(CACHE_FILE)
        except Exception:
            pass
    ACCOUNT_NAME_MAP.clear()
    EXCHANGE_MAP.clear()
    CONTRACT_CN_NAME_MAP.clear()
    COMMODITY_CN_MAP.clear()

# -------------------- 全局对照表 --------------------
ACCOUNT_NAME_MAP: Dict[str, str] = {}         # 6600xxxx -> 账号姓名
EXCHANGE_MAP: Dict[str, str] = {}             # 前缀 -> 交易所
CONTRACT_CN_NAME_MAP: Dict[str, str] = {}     # 前缀 -> 中文名称（供解析备用；不在表格展示）
COMMODITY_CN_MAP: Dict[str, str] = {}         # 中文名/别名 -> 前缀

DEFAULT_CCY = "CNY"
DEFAULT_REF = "1"

# -------------------- 正则 --------------------
RE_REF = re.compile(r'\b(?P<acc4>\d{4})\.(?P<seq>\d+)\b')
RE_CONTRACT_ANY = re.compile(r'([A-Za-z]{1,2}\s*\d{2,4})(?:\.(?P<suf>[A-Za-z]+))?')
RE_EN_LEG = re.compile(
    r'\b(?P<act>sold|sell|bot|buy|filled)\b'
    r'(?:\s+(?P<qty>\d+)\s*[x×])?'
    r'(?:\s+(?P<code>[A-Za-z]{1,2}\s*\d{2,4}))?'
    r'(?:\s*(?:at)\s*(?P<price>\d+(?:\.\d+)?))?',
    re.IGNORECASE
)

# -------------------- 数据结构 --------------------
@dataclass
class OrderRow:
    编号: int
    成交时间: str
    账号姓名: str
    交易账号: str
    买卖: str
    开平: str
    委托数量: int
    交易所: str
    合约: str
    成交均价: float
    状态: str
    成交数量: int
    币种: str
    参考: str
    录单交易员: str
    检查交易员: str
    参考编号: str
    对冲方: str

# -------------------- Excel I/O（对照表导入/模板导出） --------------------
def _require_pandas():
    if pd is None:
        raise RuntimeError("缺少 pandas，请先：pip install pandas openpyxl")

def load_account_map_from_excel(path: str) -> Dict[str, str]:
    _require_pandas()
    df = pd.read_excel(path)
    cols = {str(c).strip().lower(): c for c in df.columns}
    acc_col = cols.get("交易账号") or cols.get("account") or cols.get("后四位")
    name_col = cols.get("账号姓名") or cols.get("name")
    if not acc_col or not name_col:
        raise ValueError("账号对照表需包含列：‘交易账号/account/后四位’ 与 ‘账号姓名/name’。")
    mp = {}
    for _, r in df.iterrows():
        acc = str(r[acc_col]).strip()
        name = str(r[name_col]).strip()
        if not acc or not name or acc.lower() == 'nan': continue
        acc = re.sub(r'\D', '', acc)
        if len(acc) == 4: acc = "6600" + acc
        if len(acc) < 6:  continue
        mp[acc] = name
    return mp

def load_exchange_map_from_excel(path: str) -> Tuple[Dict[str,str], Dict[str,str], Dict[str,str]]:
    _require_pandas()
    df = pd.read_excel(path)
    cols = {str(c).strip().lower(): c for c in df.columns}
    pre_col = cols.get("合约前缀") or cols.get("prefix")
    ex_col  = cols.get("交易所") or cols.get("exchange")
    cn_col  = cols.get("中文名称") or cols.get("中文名") or cols.get("chinese_name")
    if not pre_col or not ex_col:
        raise ValueError("合约对照表需包含列：‘合约前缀/prefix’ 与 ‘交易所/exchange’。")
    ex_mp, cn_mp, cn_inv = {}, {}, {}
    for _, r in df.iterrows():
        pre = str(r[pre_col]).strip().upper()
        ex  = str(r[ex_col]).strip().upper()
        if not re.fullmatch(r'[A-Z]{1,2}', pre) or not ex: continue
        ex = {"SHF":"SHFE","SHFE":"SHFE","DCE":"DCE","CZCE":"CZCE","INE":"INE"}.get(ex, ex)
        ex_mp[pre] = ex
        if cn_col:
            cn = str(r[cn_col]).strip()
            if cn and cn.lower()!='nan':
                cn_mp[pre] = cn
                cn_inv.setdefault(cn, pre)
    return ex_mp, cn_mp, cn_inv

def load_commodity_map_from_excel(path: str) -> Dict[str, str]:
    _require_pandas()
    df = pd.read_excel(path)
    cols = {str(c).strip().lower(): c for c in df.columns}
    name_col = cols.get("品种") or cols.get("name") or cols.get("中文名称") or cols.get("中文名") or cols.get("alias")
    pre_col  = cols.get("前缀") or cols.get("prefix")
    if not name_col or not pre_col:
        raise ValueError("品种别名对照表需包含列：‘品种/中文名称/alias(name)’ 与 ‘前缀(prefix)’。")
    mp = {}
    for _, r in df.iterrows():
        n = str(r[name_col]).strip()
        p = str(r[pre_col]).strip().upper()
        if n and re.fullmatch(r'[A-Z]{1,2}', p):
            mp[n] = p
    return mp

def save_mapping_templates():
    _require_pandas()
    save_dir = filedialog.askdirectory(title="选择保存对照表模板的文件夹")
    if not save_dir: return

    acc_path = os.path.join(save_dir, "账号对照表模板.xlsx")
    exch_path = os.path.join(save_dir, "合约对照表模板.xlsx")
    comm_path = os.path.join(save_dir, "品种别名对照表模板.xlsx")

    acc_demo = pd.DataFrame({
        "交易账号": ["66003001","66001901"],
        "账号姓名": ["MANIFOLD","NEWPORT SPC HUDSONFORTUNE"]
    })
    exch_demo = pd.DataFrame({
        "合约前缀": ["JM","EG","CU","AG","RU","SN","FG","PS","AU","JD","PG","SH"],
        "交易所":   ["DCE","DCE","SHFE","SHFE","SHFE","SHFE","CZCE","CZCE","SHFE","DCE","DCE","SHFE"],
        "中文名称": ["焦煤","乙二醇","铜","白银","橡胶","锡","玻璃","聚丙烯","黄金","鸡蛋","液化气","烧碱"]
    })
    comm_demo = pd.DataFrame({
        "品种": ["焦炭","焦煤","玻璃","鸡蛋","烧碱","液化气","黄金","白银"],
        "前缀": ["J","JM","FG","JD","SH","PG","AU","AG"]
    })

    acc_demo.to_excel(acc_path, index=False)
    exch_demo.to_excel(exch_path, index=False)
    comm_demo.to_excel(comm_path, index=False)

    messagebox.showinfo(
        "完成",
        "已生成 3 个对照表模板：\n"
        f"• {acc_path}\n"
        f"• {exch_path}\n"
        f"• {comm_path}\n\n"
        "请按需要填写后，在软件中点击“导入账号对照表 / 导入合约对照表 / 导入品种别名对照表（可选）”。"
    )

# -------------------- 工具函数（标准化/识别） --------------------
def _year_for_two_digit() -> str:
    return "25"  # 两位年默认补 2025

def expand_2digit(code: str) -> str:
    code = code.strip().upper().replace(" ", "")
    m = re.fullmatch(r'([A-Z]{1,2})(\d{2})', code)
    if not m: return code
    return f"{m.group(1)}{_year_for_two_digit()}{m.group(2)}"

def normalize_ex_suffix(suf: str) -> str:
    return {"SHF":"SHFE","SHFE":"SHFE","DCE":"DCE","CZCE":"CZCE","INE":"INE"}.get(suf.upper(), suf.upper())

def normalize_contract_token(token: str) -> Tuple[str,str,str]:
    m = RE_CONTRACT_ANY.search(token)
    if not m: return "", "", ""
    code = expand_2digit(m.group(1))
    pre = ''.join([c for c in code if c.isalpha()])
    suf = m.group('suf')
    exch = normalize_ex_suffix(suf) if suf else EXCHANGE_MAP.get(pre, "")
    return code, pre, exch

def find_ref(text: str) -> str:
    m = RE_REF.search(text)
    return m.group(0) if m else ""

def find_account(text: str) -> Tuple[str,str]:
    m = re.search(r'\b(6600\d{2,6})\b', text)
    if m:
        acc = m.group(1)
        return acc, ACCOUNT_NAME_MAP.get(acc, "")
    m2 = RE_REF.search(text)
    if m2:
        acc = "6600" + m2.group("acc4")
        return acc, ACCOUNT_NAME_MAP.get(acc, "")
    return "", ""

def qty_from_text(text: str) -> int:
    m = re.search(r'数量[:：]\s*(\d+)', text)
    if m: return int(m.group(1))
    m = re.search(r'(\d+)\s*(手|张)\b', text)
    if m: return int(m.group(1))
    m = re.search(r'\b(?:buy|bot|sell|sold)\b\s+(\d+)\s*[x×]?', text, re.I)
    if m: return int(m.group(1))
    return 0

def dir_from_field(text: str, scope: Optional[str]=None) -> Tuple[Optional[str], Optional[str]]:
    if scope in ("第一腿","第二腿"):
        m = re.search(rf'{scope}\s*[:：].*?方向\s*[:：]\s*(买入开仓|买入平仓|卖出开仓|卖出平仓)', text)
    else:
        m = re.search(r'方向\s*[:：]\s*(买入开仓|买入平仓|卖出开仓|卖出平仓)', text)
    if not m: return None, None
    w = m.group(1)
    if "买入开仓" in w:  return "B","Open"
    if "买入平仓" in w:  return "B","Close"
    if "卖出开仓" in w:  return "S","Open"
    if "卖出平仓" in w:  return "S","Close"
    return None, None

def dir_from_free(text: str) -> Tuple[Optional[str], Optional[str]]:
    t = text.lower()
    side = None; oc = None
    if "开多" in text or "买开" in text: side, oc = "B", "Open"
    if "开空" in text or "卖开" in text: side, oc = "S", "Open"
    if "卖平" in text: side, oc = "S", "Close"
    if "买平" in text: side, oc = "B", "Close"
    if side is None:
        if "buy" in t or "bot" in t: side = "B"
        if "sell" in t or "sold" in t: side = "S"
    if oc is None:
        if "close" in t: oc = "Close"
        if "open" in t: oc = "Open"
    return side, oc

def price_from_text(text: str) -> float:
    m = re.search(r'\bfilled\s+at\s+(\d+(?:\.\d+)?)', text, re.I)
    if m: return float(m.group(1))
    m = re.search(r'\bat\s*(\d+(?:\.\d+)?)', text, re.I)
    if m: return float(m.group(1))
    if re.search(r'价格[:：].*(盘口|最新|市价)', text): return 0.0
    m = re.search(r'价格[:：]\s*([0-9]+(?:\.[0-9]+)?)', text)
    if m: return float(m.group(1))
    return 0.0

def split_blocks(s: str) -> List[str]:
    chunks = re.split(r'(?:\r?\n){2,}', s.strip())
    out=[]
    for c in chunks:
        c=c.strip()
        if not c: continue
        parts = re.split(r'(?=^\s*\d{4}\.\d+\s*$)', c, flags=re.M)
        for p in parts:
            p=p.strip()
            if p: out.append(p)
    return out

# -------------------- 颜色：黄金比例取色（高区分） --------------------
def _hex_from_hsv(h: float, s: float, v: float) -> str:
    import colorsys
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return f"{int(r*255):02X}{int(g*255):02X}{int(b*255):02X}"

def _pastel_hex_by_index(idx: int) -> str:
    phi = 0.61803398875
    h = (idx * phi) % 1.0
    s = 0.38
    v = 0.97
    return _hex_from_hsv(h, s, v)

def account_hex_color_by_order(acc_order_index: int) -> str:
    return _pastel_hex_by_index(acc_order_index)

# -------------------- 解析核心 --------------------
def parse_block(block: str, default_dt: str, trader: str, checker: str, hedge: str) -> List[OrderRow]:
    rows: List[OrderRow] = []
    ref = find_ref(block)
    acc, acc_name = find_account(block)

    # 1) 识别“移仓”
    ro = parse_rollover(block)
    if ro:
        for side, code, qty in ro:
            pre = ''.join([c for c in code if c.isalpha()])
            exch = EXCHANGE_MAP.get(pre, "")
            price = 0.0
            status = "Working"
            rows.append(OrderRow(0, default_dt, acc_name, acc, side, "Open" if side=="B" else "Close",
                                 qty, exch, code, price, status, 0, DEFAULT_CCY, DEFAULT_REF,
                                 trader, checker, ref, hedge))
        enrich_with_english(block, rows)
        return rows

    # 2) 两腿结构
    legs_plan = []
    if ("第一腿" in block) or ("第二腿" in block):
        for scope in ("第一腿","第二腿"):
            m_code = re.search(rf'{scope}[:：]\s*([^\n\r]+)', block)
            code, pre, ex = normalize_contract_token(m_code.group(1)) if m_code else ("","","")
            side, oc = dir_from_field(block, scope=scope)
            if side is None or oc is None:
                d = re.search(rf'{scope}[:：].*?(买入开仓|买入平仓|卖出开仓|卖出平仓)', block)
                if d:
                    w=d.group(1)
                    side, oc = ("B","Open") if "买入开仓" in w else ("B","Close") if "买入平仓" in w else ("S","Open") if "卖出开仓" in w else ("S","Close")
            q = re.search(rf'{scope}[:：].*?数量[:：]\s*(\d+)', block)
            qty = int(q.group(1)) if q else qty_from_text(block)
            exch = ex or (EXCHANGE_MAP.get(pre, "") if pre else "")
            if code:
                legs_plan.append({"code":code, "pre":pre, "exch":exch, "side":side or "", "oc":oc or "", "qty":qty})
        rows = [OrderRow(0, default_dt, acc_name, acc, lp["side"], lp["oc"], lp["qty"],
                         lp["exch"], lp["code"], 0.0, "Working", 0, DEFAULT_CCY, DEFAULT_REF,
                         trader, checker, ref, hedge) for lp in legs_plan]
        enrich_with_english(block, rows)
        return rows

    # 3) 单腿
    side, oc = dir_from_field(block)
    if side is None or oc is None:
        side, oc = dir_from_free(block)
    m_en_code = re.search(r'\b(?:sell|sold|buy|bot)\b\s+\d+\s*[x×]?\s+([A-Za-z]{1,2}\s*\d{2,4})', block, re.I)
    if m_en_code:
        code, pre, exch = normalize_contract_token(m_en_code.group(1))
    else:
        code, pre, exch = normalize_contract_token(block)
    if not code:
        for cn, p in {**{v:k for k,v in CONTRACT_CN_NAME_MAP.items()}, **COMMODITY_CN_MAP}.items():
            if cn in block:
                mmm = re.search(rf'{re.escape(cn)}[^\d]*?(\d{{2,4}})', block)
                if mmm:
                    code = expand_2digit(p + mmm.group(1)); pre = p; exch = EXCHANGE_MAP.get(pre,""); break
    qty = qty_from_text(block)
    price = price_from_text(block)
    status = "Working" if price==0 else "Filled"
    done = 0 if status=="Working" else qty
    if code:
        rows.append(OrderRow(0, default_dt, acc_name, acc, side or "", oc or "", qty,
                             exch, code, price, status, done, DEFAULT_CCY, DEFAULT_REF,
                             trader, checker, ref, hedge))
    return rows

def parse_rollover(text: str) -> Optional[List[Tuple[str,str,int]]]:
    name2pre = {**{v:k for k,v in CONTRACT_CN_NAME_MAP.items()}, **COMMODITY_CN_MAP}
    for cn, pre in name2pre.items():
        if cn in text:
            m = re.search(rf'{re.escape(cn)}[A-Za-z]{{0,2}}?(\d{{2,4}}).*?(多头|空头)\s*移仓\s*(\d+)\s*手.*?到\s*([A-Za-z]{{0,2}})?(\d{{2,4}})', text)
            if m:
                old = expand_2digit(pre + m.group(1))
                new = expand_2digit((m.group(4) or pre) + m.group(5))
                qty = int(m.group(3))
                if "多头" in m.group(2):  # 卖旧买新
                    return [("S", old, qty), ("B", new, qty)]
                else:                    # 买旧卖新
                    return [("B", old, qty), ("S", new, qty)]
    return None

def enrich_with_english(block: str, rows: List[OrderRow]) -> None:
    if not rows: return
    en_legs = []
    for m in RE_EN_LEG.finditer(block):
        act = m.group('act').lower()
        side = "S" if act in ("sold","sell") else "B" if act in ("buy","bot") else None
        qty  = int(m.group('qty')) if m.group('qty') else None
        code = m.group('code').upper().replace(" ","") if m.group('code') else None
        price= float(m.group('price')) if m.group('price') else None
        if code:
            code = expand_2digit(code)
        if side or qty or price or code:
            en_legs.append({"side":side,"qty":qty,"code":code,"price":price})

    if not en_legs:
        price = price_from_text(block)
        if price>0:
            for r in rows:
                r.成交均价 = price; r.状态="Filled"; r.成交数量 = r.委托数量
        return

    used = set()
    # 1) code 精确
    for e in en_legs:
        if not e["code"]: continue
        matches = [ri for ri,r in enumerate(rows) if r.合约.upper()==e["code"].upper()]
        if len(matches)==1:
            ri = matches[0]
            apply_en_to_row(rows[ri], e); used.add(ri)
    # 2) 方向+就近
    for e in en_legs:
        cand = []
        for ri,r in enumerate(rows):
            if ri in used: continue
            if e["side"] and r.买卖 and e["side"]!=r.买卖: continue
            cand.append(ri)
        if len(cand)==1:
            apply_en_to_row(rows[cand[0]], e); used.add(cand[0])
    # 3) 顺序填充
    for e in en_legs:
        remaining = [ri for ri in range(len(rows)) if ri not in used]
        if not remaining: break
        apply_en_to_row(rows[remaining[0]], e); used.add(remaining[0])

def apply_en_to_row(r: OrderRow, e: Dict):
    if e.get("side") and not r.买卖:
        r.买卖 = e["side"]
        r.开平 = r.开平 or ("Open" if r.买卖=="B" else "Close")
    if e.get("qty") and (not r.委托数量 or r.委托数量==0):
        r.委托数量 = e["qty"]
    if e.get("price"):
        r.成交均价 = e["price"]
    if r.成交均价 and r.成交均价>0:
        r.状态 = "Filled"; r.成交数量 = r.委托数量

# -------------------- GUI --------------------
class App(tk.Tk):
    def __init__(self):
        super().__init__()

        # 更好点按：缩放 + 大按钮样式
        try:
            self.tk.call('tk', 'scaling', 1.25)
        except Exception:
            pass
        style = ttk.Style(self)
        try:
            if platform.system() == "Darwin":
                style.theme_use('clam')
        except Exception:
            pass
        style.configure('Big.TButton', padding=(12, 8))

        self.title("TRS交易指令解析与导出工具")
        self.geometry("1360x920")

        # ========= 用垂直分栏，让下方预览更大 =========
        self.paned = ttk.Panedwindow(self, orient="vertical")
        self.paned.pack(fill="both", expand=True)

        # 上半区（控件 + 文本框）
        upper = ttk.Frame(self.paned)
        self.paned.add(upper, weight=1)   # 较小权重

        # 下半区（预览表）
        lower = ttk.Frame(self.paned)
        self.paned.add(lower, weight=3)   # 预览权重大

        # ---------- 上半区：公共信息 ----------
        top = ttk.Frame(upper); top.pack(fill="x", padx=12, pady=8)
        self.var_date = tk.StringVar(value=dt.date.today().strftime("%Y/%m/%d"))
        self.var_time = tk.StringVar(value="9:00")
        self.var_trader = tk.StringVar(value="Theo")
        self.var_checker = tk.StringVar(value="")
        ttk.Label(top, text="成交日期(YYYY/MM/DD):").grid(row=0,column=0,sticky="w", padx=4)
        ttk.Entry(top, textvariable=self.var_date, width=12).grid(row=0,column=1,padx=8)
        ttk.Label(top, text="成交时间(HH:MM):").grid(row=0,column=2,sticky="w", padx=4)
        ttk.Entry(top, textvariable=self.var_time, width=8).grid(row=0,column=3,padx=8)
        ttk.Label(top, text="录单交易员:").grid(row=0,column=4,sticky="w", padx=4)
        ttk.Entry(top, textvariable=self.var_trader, width=14).grid(row=0,column=5,padx=8)
        ttk.Label(top, text="检查交易员:").grid(row=0,column=6,sticky="w", padx=4)
        ttk.Entry(top, textvariable=self.var_checker, width=14).grid(row=0,column=7,padx=8)

        # 对照表导入 + 模板（第一排）
        bar1 = ttk.Frame(upper); bar1.pack(fill="x", padx=12, pady=(4,4))
        ttk.Button(bar1, text="导入账号对照表", style='Big.TButton', command=self._on_load_account)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar1, text="导入合约对照表（含中文名）", style='Big.TButton', command=self._on_load_exchange)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar1, text="导入品种别名对照表（可选）", style='Big.TButton', command=self._on_load_commodity)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar1, text="生成对照表模板", style='Big.TButton', command=self._on_save_templates)\
            .pack(side="left", padx=12, pady=2)

        # 清缓存（第二排）
        bar2 = ttk.Frame(upper); bar2.pack(fill="x", padx=12, pady=(0,6))
        ttk.Button(bar2, text="清除账号对照表缓存", style='Big.TButton', command=self._on_clear_acc_cache)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar2, text="清除合约对照表缓存", style='Big.TButton', command=self._on_clear_ex_cache)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar2, text="清除品种别名对照表缓存", style='Big.TButton', command=self._on_clear_comm_cache)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar2, text="清除全部对照表缓存", style='Big.TButton', command=self._on_clear_all_cache)\
            .pack(side="left", padx=12, pady=2)

        # 三个上手框（对冲方为输入框）——把高度调低点，给预览让位
        mid = ttk.Frame(upper); mid.pack(fill="both", expand=False, padx=12, pady=6)
        self.hedge_vars=[]; self.txts=[]
        for i in range(3):
            f = ttk.Labelframe(mid, text=f"上手{i+1}")
            f.grid(row=0,column=i,sticky="nsew",padx=8, pady=4); mid.columnconfigure(i, weight=1)
            ttk.Label(f, text="对冲方（可输入任意文字）:").pack(anchor="w", padx=8, pady=(8,0))
            v = tk.StringVar(value="")
            ttk.Entry(f, textvariable=v).pack(fill="x", padx=8, pady=6)
            txt = tk.Text(f, height=12, wrap="word")   # 原来 18 -> 12
            txt.pack(fill="both", expand=True, padx=8, pady=8)
            self.hedge_vars.append(v); self.txts.append(txt)

        # 操作区
        bot = ttk.Frame(upper); bot.pack(fill="x", padx=12, pady=6)
        ttk.Button(bot, text="添加到预览（累积）", style='Big.TButton', command=self.add_to_preview)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bot, text="清空输入框", style='Big.TButton', command=self.clear_inputs)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bot, text="删除选中行", style='Big.TButton', command=self.delete_selected)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bot, text="撤销上次添加", style='Big.TButton', command=self.undo_last_add)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bot, text="导出Excel（含颜色）", style='Big.TButton', command=self.export_excel)\
            .pack(side="left", padx=12, pady=2)

        # ---------- 下半区：预览表 ----------
        self.preview_frame = ttk.Frame(lower)
        self.preview_frame.pack(fill="both", expand=True, padx=12, pady=(6,12))
        self.tree = None
        self.rows: List[OrderRow] = []   # 累积数据
        self._acc_color_cache: Dict[str, Tuple[str,str]] = {}  # acc -> (tag, color)
        self._add_stacks: List[int] = []  # 每次 add 的行数，用于撤销
        self._acc_order: Dict[str, int] = {}  # 账号首次出现顺序

        self._build_tree()

        # 初始把分隔条往上推一些，让下方预览更大
        self.after(100, self._init_sash)

        # 启动时尝试读取缓存
        if load_mappings_cache():
            messagebox.showinfo("对照表缓存", "已自动载入上次保存的对照表，无需重新导入。")

    def _init_sash(self):
        try:
            # 将第0个分隔条位置设置为窗口高度的 40% 左右
            total_h = self.winfo_height()
            pos = int(total_h * 0.40)
            self.paned.sashpos(0, pos)
        except Exception:
            pass

    # ---------- 树表（去掉“合约中文名”列） ----------
    def _build_tree(self):
        cols = ["编号","成交时间","账号姓名","交易账号","买卖","开平","委托数量",
                "交易所","合约","成交均价","状态","成交数量","币种","参考",
                "录单交易员","检查交易员","参考编号","对冲方"]
        self.tree = ttk.Treeview(self.preview_frame, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=100 if c not in ("成交时间","账号姓名","参考编号") else 140, stretch=True)
        self.tree.pack(fill="both", expand=True)
        self.tree.bind("<Double-1>", self._begin_edit_cell)
        self.tree.bind("<F2>", self._begin_edit_cell)

    def _refresh_tree(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        for idx, r in enumerate(self.rows, start=1):
            r.编号 = idx
            vals = [r.编号,r.成交时间,r.账号姓名,r.交易账号,r.买卖,r.开平,r.委托数量,
                    r.交易所,r.合约,r.成交均价,r.状态,r.成交数量,r.币种,r.参考,
                    r.录单交易员,r.检查交易员,r.参考编号,r.对冲方]
            tag = self._tag_for_account(r.交易账号 or "unknown")
            self.tree.insert("", "end", values=vals, tags=(tag,))

    def _tag_for_account(self, acc: str) -> str:
        if acc not in self._acc_order:
            self._acc_order[acc] = len(self._acc_order)
        idx = self._acc_order[acc]
        color = f"#{_pastel_hex_by_index(idx)}"

        if acc in self._acc_color_cache:
            tag, _ = self._acc_color_cache[acc]
        else:
            tag = f"acc_{idx+1}"
            self._acc_color_cache[acc] = (tag, color)

        self.tree.tag_configure(tag, background=color)
        return tag

    # ---------- 编辑 ----------
    def _begin_edit_cell(self, event):
        region = self.tree.identify("region", event.x, event.y)
        if region != "cell": return
        row_id = self.tree.identify_row(event.y)
        col_id = self.tree.identify_column(event.x)
        if not row_id or not col_id: return
        x,y,w,h = self.tree.bbox(row_id, col_id)
        v = self.tree.set(row_id, col_id)
        self._edit_var = tk.StringVar(value=v)
        self._edit_entry = tk.Entry(self.tree, textvariable=self._edit_var)
        self._edit_entry.place(x=x,y=y,width=w,height=h)
        self._edit_entry.focus_set()
        self._edit_entry.bind("<Return>", lambda e: self._commit_edit(row_id, col_id))
        self._edit_entry.bind("<Escape>", lambda e: self._cancel_edit())

    def _commit_edit(self, row_id, col_id):
        newv = self._edit_var.get().strip()
        self.tree.set(row_id, col_id, newv)
        self._cancel_edit()
        idx = self.tree.index(row_id)
        cols = ["编号","成交时间","账号姓名","交易账号","买卖","开平","委托数量",
                "交易所","合约","成交均价","状态","成交数量","币种","参考",
                "录单交易员","检查交易员","参考编号","对冲方"]
        field = cols[int(col_id[1:])-1]
        if 0 <= idx < len(self.rows):
            tmap = OrderRow.__annotations__
            if field in tmap:
                target = tmap[field]
                try:
                    if target is int:
                        val = int(float(newv))
                    elif target is float:
                        val = float(newv)
                    else:
                        val = newv
                except:
                    val = newv
                setattr(self.rows[idx], field, val)

    def _cancel_edit(self):
        if hasattr(self, "_edit_entry") and self._edit_entry:
            self._edit_entry.destroy()
            self._edit_entry = None

    # ---------- 按钮动作 ----------
    def add_to_preview(self):
        date = self.var_date.get().strip()
        time_ = self.var_time.get().strip()
        trader = self.var_trader.get().strip()
        checker = self.var_checker.get().strip()
        dt_str = f"{date} {time_}"

        new_rows: List[OrderRow] = []
        for i in range(3):
            hedge = self.hedge_vars[i].get().strip()
            text = self.txts[i].get("1.0","end").strip()
            if not text: continue
            blocks = split_blocks(text)
            for b in blocks:
                if re.fullmatch(r'\s*\d{4}\.\d+\s*', b):
                    continue
                parsed = parse_block(b, dt_str, trader, checker, hedge)
                new_rows.extend(parsed)

        if not new_rows:
            messagebox.showinfo("提示","没有识别到有效指令。"); return

        self.rows.extend(new_rows)
        self._add_stacks.append(len(new_rows))
        self._refresh_tree()

    def clear_inputs(self):
        for t in self.txts:
            t.delete("1.0","end")

    def delete_selected(self):
        sels = self.tree.selection()
        if not sels:
            messagebox.showinfo("提示","请先在预览中选中要删除的行。"); return
        idxs = sorted([self.tree.index(s) for s in sels], reverse=True)
        for i in idxs:
            if 0 <= i < len(self.rows):
                del self.rows[i]
        self._refresh_tree()

    def undo_last_add(self):
        if not self._add_stacks:
            messagebox.showinfo("提示","没有可撤销的添加。"); return
        n = self._add_stacks.pop()
        if n>0:
            self.rows = self.rows[:-n]
            self._refresh_tree()

    def export_excel(self):
        if pd is None:
            messagebox.showwarning("缺少依赖","请先安装：pip install pandas openpyxl")
            return
        if not self.rows:
            messagebox.showinfo("提示","预览为空，无法导出。"); return

        save_path = filedialog.asksaveasfilename(
            title="保存为Excel", defaultextension=".xlsx", filetypes=[("Excel","*.xlsx")]
        )
        if not save_path: return

        # 1) 写数据（不再插入“合约中文名”）
        df = pd.DataFrame([asdict(r) for r in self.rows])
        df.to_excel(save_path, index=False)

        # 2) 给行着色（按账号首次出现顺序）
        try:
            from openpyxl import load_workbook
            from openpyxl.styles import PatternFill
        except Exception as e:
            messagebox.showwarning("提示", f"已导出纯数据，但未着色（缺少 openpyxl 或版本问题）：{e}")
            return

        wb = load_workbook(save_path)
        ws = wb.active

        header_idx = {cell.value: idx+1 for idx, cell in enumerate(ws[1])}
        acc_col = header_idx.get("交易账号")
        if not acc_col:
            wb.save(save_path)
            messagebox.showinfo("完成", f"已导出：{save_path}")
            return

        max_row = ws.max_row
        max_col = ws.max_column

        # 账号首次出现顺序
        acc_order: Dict[str, int] = {}
        order_counter = 0
        for r in range(2, max_row + 1):
            acc_val = ws.cell(row=r, column=acc_col).value or "unknown"
            if acc_val not in acc_order:
                acc_order[acc_val] = order_counter
                order_counter += 1

        fill_cache: Dict[str, PatternFill] = {}
        for r in range(2, max_row + 1):
            acc_val = ws.cell(row=r, column=acc_col).value or "unknown"
            idx = acc_order.get(acc_val, 0)
            hex_rgb = account_hex_color_by_order(idx)
            if hex_rgb not in fill_cache:
                fill_cache[hex_rgb] = PatternFill(fill_type="solid", fgColor=hex_rgb)
            fill = fill_cache[hex_rgb]
            for c in range(1, max_col + 1):
                ws.cell(row=r, column=c).fill = fill

        wb.save(save_path)
        messagebox.showinfo("完成", f"已导出并着色：{save_path}")

    # ---------- 对照表导入/清缓存 ----------
    def _on_load_account(self):
        try:
            path = filedialog.askopenfilename(title="选择【账号对照表】Excel", filetypes=[("Excel","*.xlsx *.xls")])
            if not path: return
            mp = load_account_map_from_excel(path)
        except Exception as e:
            messagebox.showerror("错误", f"读取失败：{e}"); return
        ACCOUNT_NAME_MAP.clear(); ACCOUNT_NAME_MAP.update(mp)
        save_mappings_cache()
        messagebox.showinfo("成功", f"已导入 {len(mp)} 条账号对照信息（已保存到缓存，下次自动读取）。")

    def _on_load_exchange(self):
        try:
            path = filedialog.askopenfilename(title="选择【合约对照表（含中文名）】Excel", filetypes=[("Excel","*.xlsx *.xls")])
            if not path: return
            ex_mp, cn_mp, inv = load_exchange_map_from_excel(path)
        except Exception as e:
            messagebox.showerror("错误", f"读取失败：{e}"); return
        EXCHANGE_MAP.clear(); EXCHANGE_MAP.update(ex_mp)
        CONTRACT_CN_NAME_MAP.clear(); CONTRACT_CN_NAME_MAP.update(cn_mp)
        for cn, pre in inv.items():
            COMMODITY_CN_MAP.setdefault(cn, pre)
        save_mappings_cache()
        messagebox.showinfo("成功", f"合约前缀→交易所 {len(ex_mp)} 条；前缀→中文名 {len(cn_mp)} 条（已保存到缓存）。")

    def _on_load_commodity(self):
        try:
            path = filedialog.askopenfilename(title="选择【品种别名对照表】Excel（可选）", filetypes=[("Excel","*.xlsx *.xls")])
            if not path: return
            mp = load_commodity_map_from_excel(path)
        except Exception as e:
            messagebox.showerror("错误", f"读取失败：{e}"); return
        for k,v in mp.items():
            COMMODITY_CN_MAP.setdefault(k, v)
        save_mappings_cache()
        messagebox.showinfo("成功", f"已导入 {len(mp)} 条品种别名（已保存到缓存）。")

    def _on_save_templates(self):
        try:
            save_mapping_templates()
        except Exception as e:
            messagebox.showerror("错误", f"保存模板失败：{e}")

    def _on_clear_acc_cache(self):
        clear_account_cache()
        messagebox.showinfo("完成", "已清除【账号对照表】缓存。")

    def _on_clear_ex_cache(self):
        clear_exchange_cache()
        messagebox.showinfo("完成", "已清除【合约对照表】缓存（含中文名）。")

    def _on_clear_comm_cache(self):
        clear_commodity_cache()
        messagebox.showinfo("完成", "已清除【品种别名对照表】缓存。")

    def _on_clear_all_cache(self):
        clear_all_cache()
        messagebox.showinfo("完成", "已清除全部对照表缓存。请重新导入需要的 Excel。")

if __name__ == "__main__":
    App().mainloop()
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import re
import os
import json
import platform
import datetime as dt
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

try:
    import pandas as pd
except ImportError:
    pd = None

# -------------------- 缓存路径 --------------------
CACHE_DIR = os.path.join(os.path.expanduser("~"), ".trade_parser_cache")
CACHE_FILE = os.path.join(CACHE_DIR, "mappings.json")

def ensure_cache_dir():
    os.makedirs(CACHE_DIR, exist_ok=True)

def _read_cache_file() -> dict:
    if not os.path.exists(CACHE_FILE):
        return {}
    try:
        with open(CACHE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return {}

def _write_cache_file(data: dict):
    ensure_cache_dir()
    with open(CACHE_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def save_mappings_cache():
    data = _read_cache_file()
    data["ACCOUNT_NAME_MAP"] = ACCOUNT_NAME_MAP
    data["EXCHANGE_MAP"] = EXCHANGE_MAP
    data["CONTRACT_CN_NAME_MAP"] = CONTRACT_CN_NAME_MAP
    data["COMMODITY_CN_MAP"] = COMMODITY_CN_MAP
    _write_cache_file(data)

def load_mappings_cache() -> bool:
    data = _read_cache_file()
    if not data:
        return False
    ACCOUNT_NAME_MAP.clear(); ACCOUNT_NAME_MAP.update(data.get("ACCOUNT_NAME_MAP", {}))
    EXCHANGE_MAP.clear(); EXCHANGE_MAP.update(data.get("EXCHANGE_MAP", {}))
    CONTRACT_CN_NAME_MAP.clear(); CONTRACT_CN_NAME_MAP.update(data.get("CONTRACT_CN_NAME_MAP", {}))
    COMMODITY_CN_MAP.clear(); COMMODITY_CN_MAP.update(data.get("COMMODITY_CN_MAP", {}))
    return any([ACCOUNT_NAME_MAP, EXCHANGE_MAP, CONTRACT_CN_NAME_MAP, COMMODITY_CN_MAP])

def clear_account_cache():
    data = _read_cache_file()
    data["ACCOUNT_NAME_MAP"] = {}
    _write_cache_file(data)
    ACCOUNT_NAME_MAP.clear()

def clear_exchange_cache():
    data = _read_cache_file()
    data["EXCHANGE_MAP"] = {}
    data["CONTRACT_CN_NAME_MAP"] = {}
    _write_cache_file(data)
    EXCHANGE_MAP.clear()
    CONTRACT_CN_NAME_MAP.clear()

def clear_commodity_cache():
    data = _read_cache_file()
    data["COMMODITY_CN_MAP"] = {}
    _write_cache_file(data)
    COMMODITY_CN_MAP.clear()

def clear_all_cache():
    if os.path.exists(CACHE_FILE):
        try:
            os.remove(CACHE_FILE)
        except Exception:
            pass
    ACCOUNT_NAME_MAP.clear()
    EXCHANGE_MAP.clear()
    CONTRACT_CN_NAME_MAP.clear()
    COMMODITY_CN_MAP.clear()

# -------------------- 全局对照表 --------------------
ACCOUNT_NAME_MAP: Dict[str, str] = {}         # 6600xxxx -> 账号姓名
EXCHANGE_MAP: Dict[str, str] = {}             # 前缀 -> 交易所
CONTRACT_CN_NAME_MAP: Dict[str, str] = {}     # 前缀 -> 中文名称（供解析备用；不在表格展示）
COMMODITY_CN_MAP: Dict[str, str] = {}         # 中文名/别名 -> 前缀

DEFAULT_CCY = "CNY"
DEFAULT_REF = "1"

# -------------------- 正则 --------------------
RE_REF = re.compile(r'\b(?P<acc4>\d{4})\.(?P<seq>\d+)\b')
RE_CONTRACT_ANY = re.compile(r'([A-Za-z]{1,2}\s*\d{2,4})(?:\.(?P<suf>[A-Za-z]+))?')
RE_EN_LEG = re.compile(
    r'\b(?P<act>sold|sell|bot|buy|filled)\b'
    r'(?:\s+(?P<qty>\d+)\s*[x×])?'
    r'(?:\s+(?P<code>[A-Za-z]{1,2}\s*\d{2,4}))?'
    r'(?:\s*(?:at)\s*(?P<price>\d+(?:\.\d+)?))?',
    re.IGNORECASE
)

# -------------------- 数据结构 --------------------
@dataclass
class OrderRow:
    编号: int
    成交时间: str
    账号姓名: str
    交易账号: str
    买卖: str
    开平: str
    委托数量: int
    交易所: str
    合约: str
    成交均价: float
    状态: str
    成交数量: int
    币种: str
    参考: str
    录单交易员: str
    检查交易员: str
    参考编号: str
    对冲方: str

# -------------------- Excel I/O（对照表导入/模板导出） --------------------
def _require_pandas():
    if pd is None:
        raise RuntimeError("缺少 pandas，请先：pip install pandas openpyxl")

def load_account_map_from_excel(path: str) -> Dict[str, str]:
    _require_pandas()
    df = pd.read_excel(path)
    cols = {str(c).strip().lower(): c for c in df.columns}
    acc_col = cols.get("交易账号") or cols.get("account") or cols.get("后四位")
    name_col = cols.get("账号姓名") or cols.get("name")
    if not acc_col or not name_col:
        raise ValueError("账号对照表需包含列：‘交易账号/account/后四位’ 与 ‘账号姓名/name’。")
    mp = {}
    for _, r in df.iterrows():
        acc = str(r[acc_col]).strip()
        name = str(r[name_col]).strip()
        if not acc or not name or acc.lower() == 'nan': continue
        acc = re.sub(r'\D', '', acc)
        if len(acc) == 4: acc = "6600" + acc
        if len(acc) < 6:  continue
        mp[acc] = name
    return mp

def load_exchange_map_from_excel(path: str) -> Tuple[Dict[str,str], Dict[str,str], Dict[str,str]]:
    _require_pandas()
    df = pd.read_excel(path)
    cols = {str(c).strip().lower(): c for c in df.columns}
    pre_col = cols.get("合约前缀") or cols.get("prefix")
    ex_col  = cols.get("交易所") or cols.get("exchange")
    cn_col  = cols.get("中文名称") or cols.get("中文名") or cols.get("chinese_name")
    if not pre_col or not ex_col:
        raise ValueError("合约对照表需包含列：‘合约前缀/prefix’ 与 ‘交易所/exchange’。")
    ex_mp, cn_mp, cn_inv = {}, {}, {}
    for _, r in df.iterrows():
        pre = str(r[pre_col]).strip().upper()
        ex  = str(r[ex_col]).strip().upper()
        if not re.fullmatch(r'[A-Z]{1,2}', pre) or not ex: continue
        ex = {"SHF":"SHFE","SHFE":"SHFE","DCE":"DCE","CZCE":"CZCE","INE":"INE"}.get(ex, ex)
        ex_mp[pre] = ex
        if cn_col:
            cn = str(r[cn_col]).strip()
            if cn and cn.lower()!='nan':
                cn_mp[pre] = cn
                cn_inv.setdefault(cn, pre)
    return ex_mp, cn_mp, cn_inv

def load_commodity_map_from_excel(path: str) -> Dict[str, str]:
    _require_pandas()
    df = pd.read_excel(path)
    cols = {str(c).strip().lower(): c for c in df.columns}
    name_col = cols.get("品种") or cols.get("name") or cols.get("中文名称") or cols.get("中文名") or cols.get("alias")
    pre_col  = cols.get("前缀") or cols.get("prefix")
    if not name_col or not pre_col:
        raise ValueError("品种别名对照表需包含列：‘品种/中文名称/alias(name)’ 与 ‘前缀(prefix)’。")
    mp = {}
    for _, r in df.iterrows():
        n = str(r[name_col]).strip()
        p = str(r[pre_col]).strip().upper()
        if n and re.fullmatch(r'[A-Z]{1,2}', p):
            mp[n] = p
    return mp

def save_mapping_templates():
    _require_pandas()
    save_dir = filedialog.askdirectory(title="选择保存对照表模板的文件夹")
    if not save_dir: return

    acc_path = os.path.join(save_dir, "账号对照表模板.xlsx")
    exch_path = os.path.join(save_dir, "合约对照表模板.xlsx")
    comm_path = os.path.join(save_dir, "品种别名对照表模板.xlsx")

    acc_demo = pd.DataFrame({
        "交易账号": ["66003001","66001901"],
        "账号姓名": ["MANIFOLD","NEWPORT SPC HUDSONFORTUNE"]
    })
    exch_demo = pd.DataFrame({
        "合约前缀": ["JM","EG","CU","AG","RU","SN","FG","PS","AU","JD","PG","SH"],
        "交易所":   ["DCE","DCE","SHFE","SHFE","SHFE","SHFE","CZCE","CZCE","SHFE","DCE","DCE","SHFE"],
        "中文名称": ["焦煤","乙二醇","铜","白银","橡胶","锡","玻璃","聚丙烯","黄金","鸡蛋","液化气","烧碱"]
    })
    comm_demo = pd.DataFrame({
        "品种": ["焦炭","焦煤","玻璃","鸡蛋","烧碱","液化气","黄金","白银"],
        "前缀": ["J","JM","FG","JD","SH","PG","AU","AG"]
    })

    acc_demo.to_excel(acc_path, index=False)
    exch_demo.to_excel(exch_path, index=False)
    comm_demo.to_excel(comm_path, index=False)

    messagebox.showinfo(
        "完成",
        "已生成 3 个对照表模板：\n"
        f"• {acc_path}\n"
        f"• {exch_path}\n"
        f"• {comm_path}\n\n"
        "请按需要填写后，在软件中点击“导入账号对照表 / 导入合约对照表 / 导入品种别名对照表（可选）”。"
    )

# -------------------- 工具函数（标准化/识别） --------------------
def _year_for_two_digit() -> str:
    return "25"  # 两位年默认补 2025

def expand_2digit(code: str) -> str:
    code = code.strip().upper().replace(" ", "")
    m = re.fullmatch(r'([A-Z]{1,2})(\d{2})', code)
    if not m: return code
    return f"{m.group(1)}{_year_for_two_digit()}{m.group(2)}"

def normalize_ex_suffix(suf: str) -> str:
    return {"SHF":"SHFE","SHFE":"SHFE","DCE":"DCE","CZCE":"CZCE","INE":"INE"}.get(suf.upper(), suf.upper())

def normalize_contract_token(token: str) -> Tuple[str,str,str]:
    m = RE_CONTRACT_ANY.search(token)
    if not m: return "", "", ""
    code = expand_2digit(m.group(1))
    pre = ''.join([c for c in code if c.isalpha()])
    suf = m.group('suf')
    exch = normalize_ex_suffix(suf) if suf else EXCHANGE_MAP.get(pre, "")
    return code, pre, exch

def find_ref(text: str) -> str:
    m = RE_REF.search(text)
    return m.group(0) if m else ""

def find_account(text: str) -> Tuple[str,str]:
    m = re.search(r'\b(6600\d{2,6})\b', text)
    if m:
        acc = m.group(1)
        return acc, ACCOUNT_NAME_MAP.get(acc, "")
    m2 = RE_REF.search(text)
    if m2:
        acc = "6600" + m2.group("acc4")
        return acc, ACCOUNT_NAME_MAP.get(acc, "")
    return "", ""

def qty_from_text(text: str) -> int:
    m = re.search(r'数量[:：]\s*(\d+)', text)
    if m: return int(m.group(1))
    m = re.search(r'(\d+)\s*(手|张)\b', text)
    if m: return int(m.group(1))
    m = re.search(r'\b(?:buy|bot|sell|sold)\b\s+(\d+)\s*[x×]?', text, re.I)
    if m: return int(m.group(1))
    return 0

def dir_from_field(text: str, scope: Optional[str]=None) -> Tuple[Optional[str], Optional[str]]:
    if scope in ("第一腿","第二腿"):
        m = re.search(rf'{scope}\s*[:：].*?方向\s*[:：]\s*(买入开仓|买入平仓|卖出开仓|卖出平仓)', text)
    else:
        m = re.search(r'方向\s*[:：]\s*(买入开仓|买入平仓|卖出开仓|卖出平仓)', text)
    if not m: return None, None
    w = m.group(1)
    if "买入开仓" in w:  return "B","Open"
    if "买入平仓" in w:  return "B","Close"
    if "卖出开仓" in w:  return "S","Open"
    if "卖出平仓" in w:  return "S","Close"
    return None, None

def dir_from_free(text: str) -> Tuple[Optional[str], Optional[str]]:
    t = text.lower()
    side = None; oc = None
    if "开多" in text or "买开" in text: side, oc = "B", "Open"
    if "开空" in text or "卖开" in text: side, oc = "S", "Open"
    if "卖平" in text: side, oc = "S", "Close"
    if "买平" in text: side, oc = "B", "Close"
    if side is None:
        if "buy" in t or "bot" in t: side = "B"
        if "sell" in t or "sold" in t: side = "S"
    if oc is None:
        if "close" in t: oc = "Close"
        if "open" in t: oc = "Open"
    return side, oc

def price_from_text(text: str) -> float:
    m = re.search(r'\bfilled\s+at\s+(\d+(?:\.\d+)?)', text, re.I)
    if m: return float(m.group(1))
    m = re.search(r'\bat\s*(\d+(?:\.\d+)?)', text, re.I)
    if m: return float(m.group(1))
    if re.search(r'价格[:：].*(盘口|最新|市价)', text): return 0.0
    m = re.search(r'价格[:：]\s*([0-9]+(?:\.[0-9]+)?)', text)
    if m: return float(m.group(1))
    return 0.0

def split_blocks(s: str) -> List[str]:
    chunks = re.split(r'(?:\r?\n){2,}', s.strip())
    out=[]
    for c in chunks:
        c=c.strip()
        if not c: continue
        parts = re.split(r'(?=^\s*\d{4}\.\d+\s*$)', c, flags=re.M)
        for p in parts:
            p=p.strip()
            if p: out.append(p)
    return out

# -------------------- 颜色：黄金比例取色（高区分） --------------------
def _hex_from_hsv(h: float, s: float, v: float) -> str:
    import colorsys
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return f"{int(r*255):02X}{int(g*255):02X}{int(b*255):02X}"

def _pastel_hex_by_index(idx: int) -> str:
    phi = 0.61803398875
    h = (idx * phi) % 1.0
    s = 0.38
    v = 0.97
    return _hex_from_hsv(h, s, v)

def account_hex_color_by_order(acc_order_index: int) -> str:
    return _pastel_hex_by_index(acc_order_index)

# -------------------- 解析核心 --------------------
def parse_block(block: str, default_dt: str, trader: str, checker: str, hedge: str) -> List[OrderRow]:
    rows: List[OrderRow] = []
    ref = find_ref(block)
    acc, acc_name = find_account(block)

    # 1) 识别“移仓”
    ro = parse_rollover(block)
    if ro:
        for side, code, qty in ro:
            pre = ''.join([c for c in code if c.isalpha()])
            exch = EXCHANGE_MAP.get(pre, "")
            price = 0.0
            status = "Working"
            rows.append(OrderRow(0, default_dt, acc_name, acc, side, "Open" if side=="B" else "Close",
                                 qty, exch, code, price, status, 0, DEFAULT_CCY, DEFAULT_REF,
                                 trader, checker, ref, hedge))
        enrich_with_english(block, rows)
        return rows

    # 2) 两腿结构
    legs_plan = []
    if ("第一腿" in block) or ("第二腿" in block):
        for scope in ("第一腿","第二腿"):
            m_code = re.search(rf'{scope}[:：]\s*([^\n\r]+)', block)
            code, pre, ex = normalize_contract_token(m_code.group(1)) if m_code else ("","","")
            side, oc = dir_from_field(block, scope=scope)
            if side is None or oc is None:
                d = re.search(rf'{scope}[:：].*?(买入开仓|买入平仓|卖出开仓|卖出平仓)', block)
                if d:
                    w=d.group(1)
                    side, oc = ("B","Open") if "买入开仓" in w else ("B","Close") if "买入平仓" in w else ("S","Open") if "卖出开仓" in w else ("S","Close")
            q = re.search(rf'{scope}[:：].*?数量[:：]\s*(\d+)', block)
            qty = int(q.group(1)) if q else qty_from_text(block)
            exch = ex or (EXCHANGE_MAP.get(pre, "") if pre else "")
            if code:
                legs_plan.append({"code":code, "pre":pre, "exch":exch, "side":side or "", "oc":oc or "", "qty":qty})
        rows = [OrderRow(0, default_dt, acc_name, acc, lp["side"], lp["oc"], lp["qty"],
                         lp["exch"], lp["code"], 0.0, "Working", 0, DEFAULT_CCY, DEFAULT_REF,
                         trader, checker, ref, hedge) for lp in legs_plan]
        enrich_with_english(block, rows)
        return rows

    # 3) 单腿
    side, oc = dir_from_field(block)
    if side is None or oc is None:
        side, oc = dir_from_free(block)
    m_en_code = re.search(r'\b(?:sell|sold|buy|bot)\b\s+\d+\s*[x×]?\s+([A-Za-z]{1,2}\s*\d{2,4})', block, re.I)
    if m_en_code:
        code, pre, exch = normalize_contract_token(m_en_code.group(1))
    else:
        code, pre, exch = normalize_contract_token(block)
    if not code:
        for cn, p in {**{v:k for k,v in CONTRACT_CN_NAME_MAP.items()}, **COMMODITY_CN_MAP}.items():
            if cn in block:
                mmm = re.search(rf'{re.escape(cn)}[^\d]*?(\d{{2,4}})', block)
                if mmm:
                    code = expand_2digit(p + mmm.group(1)); pre = p; exch = EXCHANGE_MAP.get(pre,""); break
    qty = qty_from_text(block)
    price = price_from_text(block)
    status = "Working" if price==0 else "Filled"
    done = 0 if status=="Working" else qty
    if code:
        rows.append(OrderRow(0, default_dt, acc_name, acc, side or "", oc or "", qty,
                             exch, code, price, status, done, DEFAULT_CCY, DEFAULT_REF,
                             trader, checker, ref, hedge))
    return rows

def parse_rollover(text: str) -> Optional[List[Tuple[str,str,int]]]:
    name2pre = {**{v:k for k,v in CONTRACT_CN_NAME_MAP.items()}, **COMMODITY_CN_MAP}
    for cn, pre in name2pre.items():
        if cn in text:
            m = re.search(rf'{re.escape(cn)}[A-Za-z]{{0,2}}?(\d{{2,4}}).*?(多头|空头)\s*移仓\s*(\d+)\s*手.*?到\s*([A-Za-z]{{0,2}})?(\d{{2,4}})', text)
            if m:
                old = expand_2digit(pre + m.group(1))
                new = expand_2digit((m.group(4) or pre) + m.group(5))
                qty = int(m.group(3))
                if "多头" in m.group(2):  # 卖旧买新
                    return [("S", old, qty), ("B", new, qty)]
                else:                    # 买旧卖新
                    return [("B", old, qty), ("S", new, qty)]
    return None

def enrich_with_english(block: str, rows: List[OrderRow]) -> None:
    if not rows: return
    en_legs = []
    for m in RE_EN_LEG.finditer(block):
        act = m.group('act').lower()
        side = "S" if act in ("sold","sell") else "B" if act in ("buy","bot") else None
        qty  = int(m.group('qty')) if m.group('qty') else None
        code = m.group('code').upper().replace(" ","") if m.group('code') else None
        price= float(m.group('price')) if m.group('price') else None
        if code:
            code = expand_2digit(code)
        if side or qty or price or code:
            en_legs.append({"side":side,"qty":qty,"code":code,"price":price})

    if not en_legs:
        price = price_from_text(block)
        if price>0:
            for r in rows:
                r.成交均价 = price; r.状态="Filled"; r.成交数量 = r.委托数量
        return

    used = set()
    # 1) code 精确
    for e in en_legs:
        if not e["code"]: continue
        matches = [ri for ri,r in enumerate(rows) if r.合约.upper()==e["code"].upper()]
        if len(matches)==1:
            ri = matches[0]
            apply_en_to_row(rows[ri], e); used.add(ri)
    # 2) 方向+就近
    for e in en_legs:
        cand = []
        for ri,r in enumerate(rows):
            if ri in used: continue
            if e["side"] and r.买卖 and e["side"]!=r.买卖: continue
            cand.append(ri)
        if len(cand)==1:
            apply_en_to_row(rows[cand[0]], e); used.add(cand[0])
    # 3) 顺序填充
    for e in en_legs:
        remaining = [ri for ri in range(len(rows)) if ri not in used]
        if not remaining: break
        apply_en_to_row(rows[remaining[0]], e); used.add(remaining[0])

def apply_en_to_row(r: OrderRow, e: Dict):
    if e.get("side") and not r.买卖:
        r.买卖 = e["side"]
        r.开平 = r.开平 or ("Open" if r.买卖=="B" else "Close")
    if e.get("qty") and (not r.委托数量 or r.委托数量==0):
        r.委托数量 = e["qty"]
    if e.get("price"):
        r.成交均价 = e["price"]
    if r.成交均价 and r.成交均价>0:
        r.状态 = "Filled"; r.成交数量 = r.委托数量

# -------------------- GUI --------------------
class App(tk.Tk):
    def __init__(self):
        super().__init__()

        # 更好点按：缩放 + 大按钮样式
        try:
            self.tk.call('tk', 'scaling', 1.25)
        except Exception:
            pass
        style = ttk.Style(self)
        try:
            if platform.system() == "Darwin":
                style.theme_use('clam')
        except Exception:
            pass
        style.configure('Big.TButton', padding=(12, 8))

        self.title("交易指令解析与导出工具（累积预览 + 对照表缓存 + 高区分颜色）")
        self.geometry("1360x920")

        # ========= 用垂直分栏，让下方预览更大 =========
        self.paned = ttk.Panedwindow(self, orient="vertical")
        self.paned.pack(fill="both", expand=True)

        # 上半区（控件 + 文本框）
        upper = ttk.Frame(self.paned)
        self.paned.add(upper, weight=1)   # 较小权重

        # 下半区（预览表）
        lower = ttk.Frame(self.paned)
        self.paned.add(lower, weight=3)   # 预览权重大

        # ---------- 上半区：公共信息 ----------
        top = ttk.Frame(upper); top.pack(fill="x", padx=12, pady=8)
        self.var_date = tk.StringVar(value=dt.date.today().strftime("%Y/%m/%d"))
        self.var_time = tk.StringVar(value="9:00")
        self.var_trader = tk.StringVar(value="Theo")
        self.var_checker = tk.StringVar(value="")
        ttk.Label(top, text="成交日期(YYYY/MM/DD):").grid(row=0,column=0,sticky="w", padx=4)
        ttk.Entry(top, textvariable=self.var_date, width=12).grid(row=0,column=1,padx=8)
        ttk.Label(top, text="成交时间(HH:MM):").grid(row=0,column=2,sticky="w", padx=4)
        ttk.Entry(top, textvariable=self.var_time, width=8).grid(row=0,column=3,padx=8)
        ttk.Label(top, text="录单交易员:").grid(row=0,column=4,sticky="w", padx=4)
        ttk.Entry(top, textvariable=self.var_trader, width=14).grid(row=0,column=5,padx=8)
        ttk.Label(top, text="检查交易员:").grid(row=0,column=6,sticky="w", padx=4)
        ttk.Entry(top, textvariable=self.var_checker, width=14).grid(row=0,column=7,padx=8)

        # 对照表导入 + 模板（第一排）
        bar1 = ttk.Frame(upper); bar1.pack(fill="x", padx=12, pady=(4,4))
        ttk.Button(bar1, text="导入账号对照表", style='Big.TButton', command=self._on_load_account)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar1, text="导入合约对照表（含中文名）", style='Big.TButton', command=self._on_load_exchange)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar1, text="导入品种别名对照表（可选）", style='Big.TButton', command=self._on_load_commodity)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar1, text="生成对照表模板", style='Big.TButton', command=self._on_save_templates)\
            .pack(side="left", padx=12, pady=2)

        # 清缓存（第二排）
        bar2 = ttk.Frame(upper); bar2.pack(fill="x", padx=12, pady=(0,6))
        ttk.Button(bar2, text="清除账号对照表缓存", style='Big.TButton', command=self._on_clear_acc_cache)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar2, text="清除合约对照表缓存", style='Big.TButton', command=self._on_clear_ex_cache)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar2, text="清除品种别名对照表缓存", style='Big.TButton', command=self._on_clear_comm_cache)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bar2, text="清除全部对照表缓存", style='Big.TButton', command=self._on_clear_all_cache)\
            .pack(side="left", padx=12, pady=2)

        # 三个上手框（对冲方为输入框）——把高度调低点，给预览让位
        mid = ttk.Frame(upper); mid.pack(fill="both", expand=False, padx=12, pady=6)
        self.hedge_vars=[]; self.txts=[]
        for i in range(3):
            f = ttk.Labelframe(mid, text=f"上手{i+1}")
            f.grid(row=0,column=i,sticky="nsew",padx=8, pady=4); mid.columnconfigure(i, weight=1)
            ttk.Label(f, text="对冲方（可输入任意文字）:").pack(anchor="w", padx=8, pady=(8,0))
            v = tk.StringVar(value="")
            ttk.Entry(f, textvariable=v).pack(fill="x", padx=8, pady=6)
            txt = tk.Text(f, height=12, wrap="word")   # 原来 18 -> 12
            txt.pack(fill="both", expand=True, padx=8, pady=8)
            self.hedge_vars.append(v); self.txts.append(txt)

        # 操作区
        bot = ttk.Frame(upper); bot.pack(fill="x", padx=12, pady=6)
        ttk.Button(bot, text="添加到预览（累积）", style='Big.TButton', command=self.add_to_preview)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bot, text="清空输入框", style='Big.TButton', command=self.clear_inputs)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bot, text="删除选中行", style='Big.TButton', command=self.delete_selected)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bot, text="撤销上次添加", style='Big.TButton', command=self.undo_last_add)\
            .pack(side="left", padx=6, pady=2)
        ttk.Button(bot, text="导出Excel（含颜色）", style='Big.TButton', command=self.export_excel)\
            .pack(side="left", padx=12, pady=2)

        # ---------- 下半区：预览表 ----------
        self.preview_frame = ttk.Frame(lower)
        self.preview_frame.pack(fill="both", expand=True, padx=12, pady=(6,12))
        self.tree = None
        self.rows: List[OrderRow] = []   # 累积数据
        self._acc_color_cache: Dict[str, Tuple[str,str]] = {}  # acc -> (tag, color)
        self._add_stacks: List[int] = []  # 每次 add 的行数，用于撤销
        self._acc_order: Dict[str, int] = {}  # 账号首次出现顺序

        self._build_tree()

        # 初始把分隔条往上推一些，让下方预览更大
        self.after(100, self._init_sash)

        # 启动时尝试读取缓存
        if load_mappings_cache():
            messagebox.showinfo("对照表缓存", "已自动载入上次保存的对照表，无需重新导入。")

    def _init_sash(self):
        try:
            # 将第0个分隔条位置设置为窗口高度的 40% 左右
            total_h = self.winfo_height()
            pos = int(total_h * 0.40)
            self.paned.sashpos(0, pos)
        except Exception:
            pass

    # ---------- 树表（去掉“合约中文名”列） ----------
    def _build_tree(self):
        cols = ["编号","成交时间","账号姓名","交易账号","买卖","开平","委托数量",
                "交易所","合约","成交均价","状态","成交数量","币种","参考",
                "录单交易员","检查交易员","参考编号","对冲方"]
        self.tree = ttk.Treeview(self.preview_frame, columns=cols, show="headings")
        for c in cols:
            self.tree.heading(c, text=c)
            self.tree.column(c, width=100 if c not in ("成交时间","账号姓名","参考编号") else 140, stretch=True)
        self.tree.pack(fill="both", expand=True)
        self.tree.bind("<Double-1>", self._begin_edit_cell)
        self.tree.bind("<F2>", self._begin_edit_cell)

    def _refresh_tree(self):
        for i in self.tree.get_children():
            self.tree.delete(i)
        for idx, r in enumerate(self.rows, start=1):
            r.编号 = idx
            vals = [r.编号,r.成交时间,r.账号姓名,r.交易账号,r.买卖,r.开平,r.委托数量,
                    r.交易所,r.合约,r.成交均价,r.状态,r.成交数量,r.币种,r.参考,
                    r.录单交易员,r.检查交易员,r.参考编号,r.对冲方]
            tag = self._tag_for_account(r.交易账号 or "unknown")
            self.tree.insert("", "end", values=vals, tags=(tag,))

    def _tag_for_account(self, acc: str) -> str:
        if acc not in self._acc_order:
            self._acc_order[acc] = len(self._acc_order)
        idx = self._acc_order[acc]
        color = f"#{_pastel_hex_by_index(idx)}"

        if acc in self._acc_color_cache:
            tag, _ = self._acc_color_cache[acc]
        else:
            tag = f"acc_{idx+1}"
            self._acc_color_cache[acc] = (tag, color)

        self.tree.tag_configure(tag, background=color)
        return tag

    # ---------- 编辑 ----------
    def _begin_edit_cell(self, event):
        region = self.tree.identify("region", event.x, event.y)
        if region != "cell": return
        row_id = self.tree.identify_row(event.y)
        col_id = self.tree.identify_column(event.x)
        if not row_id or not col_id: return
        x,y,w,h = self.tree.bbox(row_id, col_id)
        v = self.tree.set(row_id, col_id)
        self._edit_var = tk.StringVar(value=v)
        self._edit_entry = tk.Entry(self.tree, textvariable=self._edit_var)
        self._edit_entry.place(x=x,y=y,width=w,height=h)
        self._edit_entry.focus_set()
        self._edit_entry.bind("<Return>", lambda e: self._commit_edit(row_id, col_id))
        self._edit_entry.bind("<Escape>", lambda e: self._cancel_edit())

    def _commit_edit(self, row_id, col_id):
        newv = self._edit_var.get().strip()
        self.tree.set(row_id, col_id, newv)
        self._cancel_edit()
        idx = self.tree.index(row_id)
        cols = ["编号","成交时间","账号姓名","交易账号","买卖","开平","委托数量",
                "交易所","合约","成交均价","状态","成交数量","币种","参考",
                "录单交易员","检查交易员","参考编号","对冲方"]
        field = cols[int(col_id[1:])-1]
        if 0 <= idx < len(self.rows):
            tmap = OrderRow.__annotations__
            if field in tmap:
                target = tmap[field]
                try:
                    if target is int:
                        val = int(float(newv))
                    elif target is float:
                        val = float(newv)
                    else:
                        val = newv
                except:
                    val = newv
                setattr(self.rows[idx], field, val)

    def _cancel_edit(self):
        if hasattr(self, "_edit_entry") and self._edit_entry:
            self._edit_entry.destroy()
            self._edit_entry = None

    # ---------- 按钮动作 ----------
    def add_to_preview(self):
        date = self.var_date.get().strip()
        time_ = self.var_time.get().strip()
        trader = self.var_trader.get().strip()
        checker = self.var_checker.get().strip()
        dt_str = f"{date} {time_}"

        new_rows: List[OrderRow] = []
        for i in range(3):
            hedge = self.hedge_vars[i].get().strip()
            text = self.txts[i].get("1.0","end").strip()
            if not text: continue
            blocks = split_blocks(text)
            for b in blocks:
                if re.fullmatch(r'\s*\d{4}\.\d+\s*', b):
                    continue
                parsed = parse_block(b, dt_str, trader, checker, hedge)
                new_rows.extend(parsed)

        if not new_rows:
            messagebox.showinfo("提示","没有识别到有效指令。"); return

        self.rows.extend(new_rows)
        self._add_stacks.append(len(new_rows))
        self._refresh_tree()

    def clear_inputs(self):
        for t in self.txts:
            t.delete("1.0","end")

    def delete_selected(self):
        sels = self.tree.selection()
        if not sels:
            messagebox.showinfo("提示","请先在预览中选中要删除的行。"); return
        idxs = sorted([self.tree.index(s) for s in sels], reverse=True)
        for i in idxs:
            if 0 <= i < len(self.rows):
                del self.rows[i]
        self._refresh_tree()

    def undo_last_add(self):
        if not self._add_stacks:
            messagebox.showinfo("提示","没有可撤销的添加。"); return
        n = self._add_stacks.pop()
        if n>0:
            self.rows = self.rows[:-n]
            self._refresh_tree()

    def export_excel(self):
        if pd is None:
            messagebox.showwarning("缺少依赖","请先安装：pip install pandas openpyxl")
            return
        if not self.rows:
            messagebox.showinfo("提示","预览为空，无法导出。"); return

        save_path = filedialog.asksaveasfilename(
            title="保存为Excel", defaultextension=".xlsx", filetypes=[("Excel","*.xlsx")]
        )
        if not save_path: return

        # 1) 写数据（不再插入“合约中文名”）
        df = pd.DataFrame([asdict(r) for r in self.rows])
        df.to_excel(save_path, index=False)

        # 2) 给行着色（按账号首次出现顺序）
        try:
            from openpyxl import load_workbook
            from openpyxl.styles import PatternFill
        except Exception as e:
            messagebox.showwarning("提示", f"已导出纯数据，但未着色（缺少 openpyxl 或版本问题）：{e}")
            return

        wb = load_workbook(save_path)
        ws = wb.active

        header_idx = {cell.value: idx+1 for idx, cell in enumerate(ws[1])}
        acc_col = header_idx.get("交易账号")
        if not acc_col:
            wb.save(save_path)
            messagebox.showinfo("完成", f"已导出：{save_path}")
            return

        max_row = ws.max_row
        max_col = ws.max_column

        # 账号首次出现顺序
        acc_order: Dict[str, int] = {}
        order_counter = 0
        for r in range(2, max_row + 1):
            acc_val = ws.cell(row=r, column=acc_col).value or "unknown"
            if acc_val not in acc_order:
                acc_order[acc_val] = order_counter
                order_counter += 1

        fill_cache: Dict[str, PatternFill] = {}
        for r in range(2, max_row + 1):
            acc_val = ws.cell(row=r, column=acc_col).value or "unknown"
            idx = acc_order.get(acc_val, 0)
            hex_rgb = account_hex_color_by_order(idx)
            if hex_rgb not in fill_cache:
                fill_cache[hex_rgb] = PatternFill(fill_type="solid", fgColor=hex_rgb)
            fill = fill_cache[hex_rgb]
            for c in range(1, max_col + 1):
                ws.cell(row=r, column=c).fill = fill

        wb.save(save_path)
        messagebox.showinfo("完成", f"已导出并着色：{save_path}")

    # ---------- 对照表导入/清缓存 ----------
    def _on_load_account(self):
        try:
            path = filedialog.askopenfilename(title="选择【账号对照表】Excel", filetypes=[("Excel","*.xlsx *.xls")])
            if not path: return
            mp = load_account_map_from_excel(path)
        except Exception as e:
            messagebox.showerror("错误", f"读取失败：{e}"); return
        ACCOUNT_NAME_MAP.clear(); ACCOUNT_NAME_MAP.update(mp)
        save_mappings_cache()
        messagebox.showinfo("成功", f"已导入 {len(mp)} 条账号对照信息（已保存到缓存，下次自动读取）。")

    def _on_load_exchange(self):
        try:
            path = filedialog.askopenfilename(title="选择【合约对照表（含中文名）】Excel", filetypes=[("Excel","*.xlsx *.xls")])
            if not path: return
            ex_mp, cn_mp, inv = load_exchange_map_from_excel(path)
        except Exception as e:
            messagebox.showerror("错误", f"读取失败：{e}"); return
        EXCHANGE_MAP.clear(); EXCHANGE_MAP.update(ex_mp)
        CONTRACT_CN_NAME_MAP.clear(); CONTRACT_CN_NAME_MAP.update(cn_mp)
        for cn, pre in inv.items():
            COMMODITY_CN_MAP.setdefault(cn, pre)
        save_mappings_cache()
        messagebox.showinfo("成功", f"合约前缀→交易所 {len(ex_mp)} 条；前缀→中文名 {len(cn_mp)} 条（已保存到缓存）。")

    def _on_load_commodity(self):
        try:
            path = filedialog.askopenfilename(title="选择【品种别名对照表】Excel（可选）", filetypes=[("Excel","*.xlsx *.xls")])
            if not path: return
            mp = load_commodity_map_from_excel(path)
        except Exception as e:
            messagebox.showerror("错误", f"读取失败：{e}"); return
        for k,v in mp.items():
            COMMODITY_CN_MAP.setdefault(k, v)
        save_mappings_cache()
        messagebox.showinfo("成功", f"已导入 {len(mp)} 条品种别名（已保存到缓存）。")

    def _on_save_templates(self):
        try:
            save_mapping_templates()
        except Exception as e:
            messagebox.showerror("错误", f"保存模板失败：{e}")

    def _on_clear_acc_cache(self):
        clear_account_cache()
        messagebox.showinfo("完成", "已清除【账号对照表】缓存。")

    def _on_clear_ex_cache(self):
        clear_exchange_cache()
        messagebox.showinfo("完成", "已清除【合约对照表】缓存（含中文名）。")

    def _on_clear_comm_cache(self):
        clear_commodity_cache()
        messagebox.showinfo("完成", "已清除【品种别名对照表】缓存。")

    def _on_clear_all_cache(self):
        clear_all_cache()
        messagebox.showinfo("完成", "已清除全部对照表缓存。请重新导入需要的 Excel。")

if __name__ == "__main__":
    App().mainloop()


2025-09-26 14:00:21.604 Python[17048:885972] +[IMKClient subclass]: chose IMKClient_Legacy
2025-09-26 14:00:21.604 Python[17048:885972] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


: 

: 